In [27]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>


In [28]:
tests = [
    {
        "name": "Example 1",
        "mass": 12,
        "expected": 2,
    },
    {
        "name": "Example 2",
        "mass": 14,
        "expected": 2,
    },
    {
        "name": "Example 3",
        "mass": 1969,
        "expected": 654,
    },
    {
        "name": "Example 4",
        "mass": 100756,
        "expected": 33583,
    },
]


def fuell(mass: int) -> int:
    """
    Fuel required to launch a given module is based on its mass.
    Specifically, to find the fuel required for a module, take its mass,
    divide by three, round down, and subtract 2.
    """
    return mass // 3 - 2


run_tests_params(fuell, tests)

example = """
12
14
1969
100756
"""


def sum_fuell(s: str) -> int:
    return sum(fuell(int(l.strip())) for l in s.strip().splitlines())


print()
print(f"Example: {sum_fuell(example)} should be {2 + 2 + 654 + 33583}")


Test Example 1 passed, for fuell.
Test Example 2 passed, for fuell.
Test Example 3 passed, for fuell.
Test Example 4 passed, for fuell.
Success

Example: 34241 should be 34241


In [29]:
with open("../input/day1.txt") as f:
    puzzle = f.read()

print(f"Part I: {sum_fuell(puzzle)}")

Part I: 3270338


<link href="style.css" rel="stylesheet"></link>

</main>


In [30]:
tests = [
    {
        "name": "Example 1",
        "mass": 12,
        "expected": 2,
    },
    {
        "name": "Example 2",
        "mass": 14,
        "expected": 2,
    },
    {
        "name": "Example 3",
        "mass": 1969,
        "expected": 966,
    },
    {
        "name": "Example 4",
        "mass": 100756,
        "expected": 50346,
    },
]


def fuell_total(mass: int) -> int:
    total_fuell = 0
    mass = fuell(mass)

    while mass > 0:
        total_fuell += mass
        mass = fuell(mass)

    return total_fuell


run_tests_params(fuell_total, tests)

example = """
12
14
1969
100756
"""


def sum_total_fuell(s: str) -> int:
    return sum(fuell_total(int(l.strip())) for l in s.strip().splitlines())


print()
print(f"Example: {sum_total_fuell(example)} should be {2 + 2 + 966 + 50346}")


Test Example 1 passed, for fuell_total.
Test Example 2 passed, for fuell_total.
Test Example 3 passed, for fuell_total.
Test Example 4 passed, for fuell_total.
Success

Example: 51316 should be 51316


In [31]:
print(f"Part II: {sum_total_fuell(puzzle)}")

Part II: 4902650


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>4902650</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
